In [1]:
import pandas as pd
import json
import os
import numpy as np

In [2]:
# folder_path = r"D:\Projects\transport\raw_bill\week15"

# file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.xlsx')]
# raw_sheets = []

# for file_path in file_paths:
#     rs = pd.read_excel(file_path, sheet_name="sheet1")
#     raw_sheets.append(rs)

# full_sheet = pd.concat(raw_sheets)

c:\Users\Boyu\anaconda3\envs\py311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Boyu\anaconda3\envs\py311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Boyu\anaconda3\envs\py311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Boyu\anaconda3\envs\py311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
signed_time_key = "Business Time"
receiver_key = "Driver's name"
fee_key = "Total amount including tax"
post_code_key = "Postal Code"
weight_key = "CW"
verify_remark_key = "Verify Remark"


In [ ]:
file_path = r"raw_bill\week15\账单明细 (2).xlsx"
full_sheet = pd.read_excel(file_path, sheet_name="sheet1", , usecols=[signed_time_key, receiver_key, fee_key, post_code_key, weight_key, verify_remark_key])

c:\Users\Boyu\anaconda3\envs\py311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [84]:
full_sheet.keys()

Index(['Serial number', 'Business Order NO.', 'Waybill No',
       'Settlement Object Type', 'Billing Title', 'Adjudicate Reason',
       'Adjudicate Description', 'Work Order Category Line', 'Bill Date',
       'Bill number', 'Revenue And Expenditure Type', 'Settlement Object Code',
       'Settlement Object Name', 'Settlement amount',
       'Total amount including tax', 'Excluding tax amount', 'Tax category',
       'Tax rate', 'Tax amount', 'Withholding and payment on behalf of others',
       'Currency', 'Billing status', 'Settlement status', 'destination zone',
       'destination station', 'Postal Code', 'Price ID', 'Driver's name',
       'Driver Code', 'Contract Coding', 'Settlement number',
       'Settlement Subject', 'Supplier Bank Account Name',
       'Supplier Bank Account Code', 'CW', 'Upload weight',
       'Upload volume weight', 'Gross weight', 'The actual volume is heavy',
       'Verify Operator', 'Verify Time', 'Business Time', 'Bill Flow Time',
       'Create Ope

In [88]:
full_sheet.shape

(64754, 6)

In [87]:
full_sheet = full_sheet[[signed_time_key, receiver_key, fee_key, post_code_key, weight_key, verify_remark_key]]

In [332]:
# name_sheet = pd.read_csv('data/name_sheet2.csv')
city_name = input("请输入城市名称 (例如: SYD): ").upper()
if city_name.upper() == "SYD":
    name_sheet_path = 'data/name_sheet_syd.csv'
    post_prefix = "2"
elif city_name.upper() == "ADL":
    name_sheet_path = 'data/name_sheet_ade.csv'
    post_prefix = "5"
elif city_name.upper() == "BNE":
    name_sheet_path = 'data/name_sheet_bne.csv'
    post_prefix = "4"
else:
    print(f"未找到城市 {city_name} 对应的name_sheet文件，将使用默认文件")
    name_sheet_path = 'data/name_sheet2.csv'



In [333]:
name_sheet = pd.read_csv(name_sheet_path)
full_sheet[post_code_key] = full_sheet[post_code_key].astype(str)
city_sheet = full_sheet[full_sheet[post_code_key].str.startswith(post_prefix)]

In [334]:
penalty_sheet = city_sheet[city_sheet[verify_remark_key] == "adjustBill"]
raw_sheet = city_sheet[city_sheet[verify_remark_key] != "adjustBill"]

In [335]:
raw_sheet.shape

(326, 6)

In [336]:
name_sheet.shape

(13, 1)

In [337]:
penalty_sheet.shape

(0, 6)

In [338]:

# 时间格式为2025-04-13 16:05:59，需要提取日期

dates = raw_sheet[signed_time_key].apply(lambda x: str(x)[:10]).unique()
dates = np.sort(dates)
dates = [str(date)[:10] for date in dates]
folder_name = "_".join([dates[0], dates[-1]])
project_path = os.path.join("output", city_name, folder_name)
project_cache_path = os.path.join(project_path, "cache")
os.makedirs(project_path, exist_ok=True)
os.makedirs(project_cache_path, exist_ok=True)

In [339]:
fees = raw_sheet[fee_key].unique()
fees.sort()

### 获取当周汇总数据

In [340]:
raw_sheet = raw_sheet[raw_sheet[fee_key] > 0]

In [341]:
summary_sheet = raw_sheet[fee_key].value_counts().sort_index()

In [342]:
# summary_sheet = summary_sheet[summary_sheet.index > 0]

summary_sheet.name = '数量'

new_data = pd.Series([summary_sheet.sum()], index=['Total'], name='数量')  # 新行数据
summary_sheet = pd.concat([summary_sheet, new_data])  # 使用 pd.concat 添加新行
summary_sheet.index.name = '单价'

In [343]:
summary_sheet.head(20)

单价
2.2      326
Total    326
Name: 数量, dtype: int64

In [344]:
summary_sheet.to_csv(f'{project_path}/week_summary.csv')

### 获取每位派送员派送详情

In [345]:
per_person_sum = raw_sheet[receiver_key].value_counts()

In [346]:
per_person_sum.name = '总数'

In [347]:
per_person_sum = pd.DataFrame(per_person_sum)
per_person_sum = per_person_sum.reset_index()

In [348]:
import pandas as pd
import re

# 定义函数，提取公司简称、账号 ID 和账号名称
def extract_account_info(account_str):
    match = re.match(r"^([A-Z]+(?:-[A-Z]+)*)-(\w+)-(.+)$", account_str)
    if match:
        company_short = match.group(1)
        account_id = match.group(2)
        account_name = match.group(3)
        return company_short, account_id, account_name
    return None, None, account_str  # 仅有姓名时，填充 None


In [349]:
# per_person_sum['name_id'] = ["-".join(x.split("-")[:2]) for x in per_person_sum.index.tolist()]
per_person_sum[['company_short', 'account_id', 'account_name']] = per_person_sum[receiver_key].apply(lambda x: pd.Series(extract_account_info(x)))

In [350]:
len(per_person_sum.index.unique())

2

In [351]:
per_person_sum.head()

,Driver's name,总数,company_short,account_id,account_name
0,ADL-BPE-006-SUHAGI DESAI,275,ADL-BPE,006,SUHAGI DESAI
1,ADL-BPE-016-YASHKUMAR A RAVAL,51,ADL-BPE,016,YASHKUMAR A RAVAL


In [352]:
# 找到新增的名字
name_sheet[['company_short', 'account_id', 'account_name']] = name_sheet['account'].apply(lambda x: pd.Series(extract_account_info(x)))
per_person_sum["account_id_name"] = per_person_sum.apply(lambda row: 
    f"{row['account_id']}-{row['account_name']}" if row["account_id"] else row["account_name"], axis=1)
per_person_sum['account_name'] = per_person_sum['account_name'].apply(lambda x: x.strip())

name_sheet["account_id_name"] = name_sheet.apply(lambda row: 
    f"{row['account_id']}-{row['account_name']}" if row["account_id"] else row["account_name"], axis=1)

name_sheet['account_name'] = name_sheet['account_name'].apply(lambda x: x.strip())


dedup_key = 'account_name'

duplicate_mask = name_sheet.duplicated(subset=[dedup_key], keep=False)
if duplicate_mask.any():
    # 获取所有重复的记录
    duplicates = name_sheet[duplicate_mask].copy()
    
    # 按account_id_name分组，对每组进行处理
    for account_id_name, group in duplicates.groupby(dedup_key):
        if len(group) > 1:  # 如果有重复记录
            # 检查是否有BPE记录
            bpe_record = group[group['company_short'] == 'BPE']
            if not bpe_record.empty:
                # 如果有BPE记录，删除其他记录
                name_sheet = name_sheet[~((name_sheet[dedup_key] == account_id_name) & 
                                        (name_sheet['company_short'] != 'BPE'))]
            else:
                # 如果没有BPE记录，保留第一条记录
                name_sheet = name_sheet[~((name_sheet[dedup_key] == account_id_name) & 
                                        (name_sheet.index != group.index[0]))]


new_names_id = set(per_person_sum[dedup_key].tolist()) - set(name_sheet[dedup_key].tolist())
new_names_id = list(new_names_id)



In [353]:
# 添加到name_sheet
if new_names_id:  # only proceed if there are new names
    new_names = []
    for name_id in new_names_id:
        # Get the full name from per_person_sum DataFrame
        new_person_record = per_person_sum[per_person_sum[dedup_key]==name_id]
        company_short, account_id, account_name = extract_account_info(new_person_record[receiver_key].values[0])
        new_names.append({
            'account': new_person_record[receiver_key].values[0],
            'company_short': company_short,
            'account_id': account_id,
            'account_name': account_name
        })
    
    new_name_sheet = pd.DataFrame(new_names)
    name_sheet = pd.concat([name_sheet, new_name_sheet], ignore_index=True)

In [354]:
name_sheet['account'].drop_duplicates().to_csv(name_sheet_path, index=False)

In [355]:
name_sheet.head()

,account,company_short,account_id,account_name,account_id_name
0,Yajie CAO,None,None,Yajie CAO,Yajie CAO
1,Wenlong ZHOU,None,None,Wenlong ZHOU,Wenlong ZHOU
2,SIMONE FERRARI,None,None,SIMONE FERRARI,SIMONE FERRARI
3,xinzhe,None,None,xinzhe,xinzhe
4,ADL-BPE-006-SUHAGI DESAI,ADL-BPE,006,SUHAGI DESAI,006-SUHAGI DESAI


In [356]:
per_person_sum.head()

,Driver's name,总数,company_short,account_id,account_name,account_id_name
0,ADL-BPE-006-SUHAGI DESAI,275,ADL-BPE,006,SUHAGI DESAI,006-SUHAGI DESAI
1,ADL-BPE-016-YASHKUMAR A RAVAL,51,ADL-BPE,016,YASHKUMAR A RAVAL,016-YASHKUMAR A RAVAL


In [357]:
per_person_sum[per_person_sum['account_name'].str.lower()=='bimarsha kandel'].head()

,Driver's name,总数,company_short,account_id,account_name,account_id_name


In [358]:
per_person_sum['总数'].sum()

326

In [359]:
name_join_sheet.shape

(105, 11)

In [360]:
name_join_sheet = pd.merge(name_sheet, per_person_sum, left_on=dedup_key, right_on=dedup_key, how='left', suffixes=('_obs', ''), sort=False)
name_join_sheet.fillna(0, inplace=True)
name_join_sheet['总数'] = name_join_sheet['总数'].astype(int) # 将总数列转换为整数
# name_sheet = name_sheet.to_json(orient='records')
# name_sheet = json.loads(name_sheet)

In [361]:
name_join_sheet['总数'].sum()

326

In [362]:
len(name_join_sheet['account_name'].unique())


13

In [363]:
name_join_sheet.head()

,account,company_short_obs,account_id_obs,account_name,account_id_name_obs,Driver's name,总数,company_short,account_id,account_id_name
0,Yajie CAO,0,0,Yajie CAO,Yajie CAO,0,0,0,0,0
1,Wenlong ZHOU,0,0,Wenlong ZHOU,Wenlong ZHOU,0,0,0,0,0
2,SIMONE FERRARI,0,0,SIMONE FERRARI,SIMONE FERRARI,0,0,0,0,0
3,xinzhe,0,0,xinzhe,xinzhe,0,0,0,0,0
4,ADL-BPE-006-SUHAGI DESAI,ADL-BPE,006,SUHAGI DESAI,006-SUHAGI DESAI,ADL-BPE-006-SUHAGI DESAI,275,ADL-BPE,006,006-SUHAGI DESAI


In [364]:
if name_join_sheet['总数'].sum() != summary_sheet.sum()//2:
    print(f"总数不匹配: {name_join_sheet['总数'].sum()} != {summary_sheet.sum()//2}")
else:
    print(f"总数匹配: {name_join_sheet['总数'].sum()} == {summary_sheet.sum()//2}")


总数匹配: 326 == 326


In [365]:
# import tkinter as tk
# from tkinter import messagebox
# import win32gui
# import win32con

# # 数据校验
# total_deliveries = name_sheet['总数'].sum()
# total_summary = summary_sheet.sum()//2

# if total_deliveries != total_summary:
#     # 创建弹窗
#     root = tk.Tk()
#     # root.withdraw()  # 隐藏主窗口
    
#     # 显示错误消息
#     error_dialog = messagebox.showerror(
#         "数据校验错误",
#         f"数据不匹配！\n\n"
#         f"派送员总数: {total_deliveries}\n"
#         f"汇总表总数: {total_summary}\n"
#         f"差异数量: {abs(total_deliveries - total_summary)}\n\n"
#         f"请检查以下可能的原因：\n"
#         f"1. 是否有派送员数据未正确统计\n"
#         f"2. 是否有重复计算的情况\n"
#         f"3. 是否有数据丢失"
#     )
    
#     # 获取错误对话框的句柄并设置为最顶层
#     hwnd = win32gui.GetForegroundWindow()
#     win32gui.SetWindowPos(hwnd, win32con.HWND_TOPMOST, 0, 0, 0, 0, 
#                          win32con.SWP_NOMOVE | win32con.SWP_NOSIZE)
    
#     # 询问是否继续
#     confirm_dialog = messagebox.askyesno("确认", "是否继续处理？")
    
#     # # 获取确认对话框的句柄并设置为最顶层
#     # hwnd = win32gui.GetForegroundWindow()
#     # win32gui.SetWindowPos(hwnd, win32con.HWND_TOPMOST, 0, 0, 0, 0, 
#     #                      win32con.SWP_NOMOVE | win32con.SWP_NOSIZE)
    
#     if not confirm_dialog:
#         raise ValueError("用户取消处理")
    
#     root.destroy()  # 关闭窗口
# else:
#     print(f"✅ 数据校验通过！总数匹配: {total_deliveries} == {total_summary}")

In [366]:
# print(counts.to_json(force_ascii=False))
# signed_time_key = None
# receiver_key = None
# fee_key = None
raw_sheet[signed_time_key] = raw_sheet[signed_time_key].apply(lambda x: str(x)[:10])
counts = raw_sheet.groupby([receiver_key, signed_time_key, fee_key]).size().reset_index(name='记录数')

In [367]:
# 转换为嵌套 JSON
nested_json = (
    counts.reset_index()  # 将 MultiIndex 转为普通列
    .groupby(receiver_key)  # 按收派员名字分组
    .apply(lambda x: x.groupby(signed_time_key)  # 在组内按签收时间分组
           .apply(lambda y: y.set_index(fee_key)['记录数'].to_dict())  # 将计费和对应数量转为字典
           .to_dict())  # 将签收时间分组转为字典
    .to_dict()  # 将收派员编号分组转为字典
)

In [368]:
name_join_sheet['data'] = name_join_sheet['account'].apply(lambda x: nested_json.get(x, {}))

In [369]:
# columns = ["序号", "编号", "ACCOUNT"]
columns = ["序号", "ACCOUNT"]
sub_columns = [""] * len(columns)
# 定义表头
for d in dates:
    columns.append(d)
    columns.extend([""] * (len(fees) - 1))

    sub_columns.extend(fees)

columns.append("Total")
sub_columns.append("")
# 创建多级表头
header = pd.MultiIndex.from_tuples(list(zip(columns, sub_columns)))

In [370]:
name_join_sheet.head()

,account,company_short_obs,account_id_obs,account_name,account_id_name_obs,Driver's name,总数,company_short,account_id,account_id_name,data
0,Yajie CAO,0,0,Yajie CAO,Yajie CAO,0,0,0,0,0,{}
1,Wenlong ZHOU,0,0,Wenlong ZHOU,Wenlong ZHOU,0,0,0,0,0,{}
2,SIMONE FERRARI,0,0,SIMONE FERRARI,SIMONE FERRARI,0,0,0,0,0,{}
3,xinzhe,0,0,xinzhe,xinzhe,0,0,0,0,0,{}
4,ADL-BPE-006-SUHAGI DESAI,ADL-BPE,006,SUHAGI DESAI,006-SUHAGI DESAI,ADL-BPE-006-SUHAGI DESAI,275,ADL-BPE,006,006-SUHAGI DESAI,"{'2025-04-09': {2.2: 97}, '2025-04-11': {2.2: ..."


In [371]:
full_data = []

for idx, person in name_join_sheet.iterrows():
    data = [idx+1, person['account']]
    for d in dates:
        if d in person['data']:
            for f in fees:
                if f in person['data'][d]:
                    data.append(person['data'][d].get(f, ""))
                else:
                    data.append("")
        else:
            data.extend([""] * len(fees))
    data.append(person['总数'])
    full_data.append(data)

In [372]:
person['总数']

0

In [373]:
# 添加汇总行
total_row = ["Total", "总计"]
total_by_date_fee = {}  # 用于存储每个日期和费用类型的总和

# 初始化汇总数据结构
for d in dates:
    total_by_date_fee[d] = {f: 0 for f in fees}

# 计算每个日期和费用类型的总和
for person in name_join_sheet.itertuples():
    for d in dates:
        if hasattr(person, 'data') and d in person.data:
            for f in fees:
                if f in person.data[d]:
                    total_by_date_fee[d][f] += person.data[d][f]

# 构建汇总行数据
for d in dates:
    for f in fees:
        total_row.append(total_by_date_fee[d][f])

# 添加总计数量
total_row.append(name_join_sheet['总数'].sum())

# 将汇总行添加到full_data
full_data.append(total_row)

In [374]:
# 创建 DataFrame
df = pd.DataFrame(full_data, columns=header)

In [375]:
from openpyxl import load_workbook

df.to_excel(f"{project_cache_path}/output.xlsx", index=True)
# 加载并移除空行
wb = load_workbook(f"{project_cache_path}/output.xlsx")
ws = wb.active

# 删除空行（通常是第3行）
ws.delete_rows(3)
# ws.delete_cols(0)

# 保存调整后的文件
wb.save(f"{project_path}/per_person_details.xlsx")
print(f"文件已保存至 {project_path}/per_person_details.xlsx")
wb.close()

文件已保存至 output\ADL\2025-04-09_2025-04-11/per_person_details.xlsx
